In [1]:
!pip install tf-nightly

In [4]:
%cd validation
!mkdir clean
!mkdir down_grade
!mv set14-downgraded/* down_grade
!mv set5-downgraded/* down_grade
!mv Set5/* clean
!mv Set14/* clean
%cd ..

/content/validation
/content


In [5]:
!rm -R /content/validation/set14-downgraded /content/validation/set5-downgraded /content/validation/Set5 /content/validation/Set14 

In [6]:
%cd /content/train

/content/train


In [7]:
!mkdir train_y
!mv General-100/* train_y
!mv "91"/* train_y

In [8]:
!rm -R General-100/ "91"/

In [9]:
# Conversion of bmp images to jpg format, it didnt change the quality of the image 
!sudo apt install imagemagick
%cd train_y
!mogrify -format jpg *.bmp
!rm *.bmp
%cd ..
%cd ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common imagemagick-6.q16 libcupsfilters1 libcupsimage2
  libdjvulibre-text libdjvulibre21 libgs9 libgs9-common libijs-0.35
  libjbig2dec0 liblqr-1-0 libmagickcore-6.q16-3 libmagickcore-6.q16-3-extra
  libmagickwand-6.q16-3 libnetpbm10 libwmf0.2-7 netpbm poppler-data
Suggested packages:
  fonts-noto ghostscript-x imagemagick-doc autotrace cups-bsd | lpr | lprng
  enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer povray radiance
  sane-utils texlive-base-bin transfig ufraw-batch inkscape libjxr-tools
  libwmf0.2-7-gtk poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-goth

In [10]:
%cd /content/validation/clean
!mogrify -format jpg *.png
!rm *.png
%cd /content/validation/down_grade
!mogrify -format jpg *.png
!rm *.png
%cd /content

/content/validation/clean
/content/validation/down_grade
/content


In [1]:
import os
import io
import PIL
import tensorflow as tf
import numpy as np
import cv2
from glob import glob
import numpy as  np
import sys
import random
from tqdm import tqdm
import argparse
import matplotlib.pyplot as plt
import copy
from functools import partial
import multiprocessing

#Check folder function, if an required folder is not present, then it creates one
def check_folder(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    return log_dir

# Normalizes the image
def normalize(images):
    return (images.astype(np.float32)/255.0)

# Retains the image to color format
def denormalize(images):
    return np.clip(images*255.0, a_min=0.001, a_max=255.0).astype(np.uint8)

# An iterator to read, process and feed images to the network
class Trainset_Dispenser():
    def __init__(self, data_path, jpeg_quality, patch_size, batch_size):
        self.data_path = data_path
        self.jpeg_quality = jpeg_quality
        self.patch_size = patch_size
        self.batch_size = batch_size
        self.images_input, self.images_label = self.load_images()

    def load_images(self):
        #pre-load images. For example, using the train dataset, 800 images will be preloaded on the memory
        file_list = glob(os.path.join(self.data_path,"*.*"))
        input_list = []
        label_list = []
        for f in tqdm(file_list):
            # read image
            label = PIL.Image.open(f).convert('RGB')

            # compress
            buffer = io.BytesIO()
            label.save(buffer, format='jpeg', quality=self.jpeg_quality)
            input = PIL.Image.open(buffer)

            # normalization and appending
            input_list.append(normalize(np.array(input)))
            label_list.append(normalize(np.array(label)))

        return input_list, label_list

    def __iter__(self):
        return self

    def __next__(self):
        patches_input = []
        patches_label = []
        for i in range(self.batch_size):
            rand_idx = random.randint(0,len(self.images_label)-1)
            crop_y = random.randint(0, self.images_label[rand_idx].shape[0] - self.patch_size-1)
            crop_x = random.randint(0, self.images_label[rand_idx].shape[1] - self.patch_size-1)
            input_patch = self.images_input[rand_idx][crop_y:crop_y+self.patch_size, crop_x:crop_x+self.patch_size]
            label_patch = self.images_label[rand_idx][crop_y:crop_y+self.patch_size, crop_x:crop_x+self.patch_size]
            patches_input.append(input_patch)
            patches_label.append(label_patch)

        patches_input = np.array(patches_input)
        patches_label = np.array(patches_label)
        return patches_input, patches_label



class Testset_Dispenser():
    def __init__(self, data_path, jpeg_quality):
        self.data_path = data_path
        self.jpeg_quality = jpeg_quality
        self.images_input, self.images_label = self.load_images()

    def load_images(self):
        #pre-load images. For example, using the DIV2K dataset, 800 images will be preloaded on the memory
        file_list = glob(os.path.join(self.data_path,"*.*"))
        input_list = []
        label_list = []
        for f in tqdm(file_list):
            # read image
            label = PIL.Image.open(f).convert('RGB')

            # compress
            buffer = io.BytesIO()
            label.save(buffer, format='jpeg', quality=self.jpeg_quality)
            input = PIL.Image.open(buffer)

            # normalization and appending
            input_list.append(normalize(np.expand_dims(np.array(input),axis=0)))
            label_list.append(normalize(np.expand_dims(np.array(label),axis=0)))

        return input_list, label_list

    def __iter__(self):
        return zip(self.images_input,self.images_label)

In [88]:
import tensorflow as tf
import numpy as np

""" =================================================================
 ops 
================================================================= """
def L1loss(input,target):
    #return tf.reduce_sum(tf.reduce_mean(tf.abs(input - target),axis=0))
    return tf.reduce_mean(tf.abs(input - target))




""" =================================================================
 model blocks
================================================================= """
initializer = tf.initializers.VarianceScaling()

def EncoderBlock(x, activation = tf.keras.layers.LeakyReLU(alpha=0.2), nf = 1024):
    x = tf.keras.layers.Dense(256, use_bias=True)(x)
    x = activation(x)
    x = tf.keras.layers.Dense(512, use_bias=True)(x)
    x = activation(x)
    x = tf.keras.layers.Dense(nf, use_bias=True)(x)
    x = activation(x)
    return x

def DecoderBlock(x, activation = tf.keras.layers.LeakyReLU(alpha=0.2), nf = 1024):
    x = tf.keras.layers.Dense(nf, use_bias=True)(x)
    x = activation(x)
    x = tf.keras.layers.Dense(512, use_bias=True)(x)
    x = activation(x)
    x = tf.keras.layers.Dense(256, use_bias=True)(x)
    x = activation(x)
    x = tf.keras.layers.Dense(3, use_bias=True)(x)
    x = activation(x)
    return x



def ConvolutionalUnit(x, structure_type = 'classic', activation = tf.keras.layers.LeakyReLU(alpha=0.2), nf = 1024):
    residual = x

    if structure_type == "classic":
        x = tf.keras.layers.Conv2D(nf, 5, strides=1, padding='same', kernel_initializer=initializer, use_bias=True)(x)
        x = activation(x)
        x = tf.keras.layers.Add()([x, residual])

    elif structure_type == "advanced":
        x = tf.keras.layers.Conv2D(nf, 5, strides=1, padding='same', kernel_initializer=initializer, use_bias=True)(x)
        x = activation(x)
        x = tf.keras.layers.Conv2D(nf, 5, strides=1, padding='same', kernel_initializer=initializer, use_bias=True)(x)
        x = tf.keras.layers.Lambda(lambda x: x * 0.1)(x)
        x = tf.keras.layers.Add()([x, residual])

    return x


def S_Net_progressiveskip(channels = 3, num_metrics=3 , structure_type='classic'):
    inputs = tf.keras.layers.Input(shape=[48, 48, channels])
    encoder = EncoderBlock(inputs)
    convolution_units = [ConvolutionalUnit(encoder)]
    for _ in range(1, num_metrics):
        convolution_units.append(ConvolutionalUnit( convolution_units[-1], structure_type = structure_type))

    decoders = []
    for e,cu in enumerate(convolution_units):
        decoders.append(tf.keras.layers.Add()([DecoderBlock(cu), inputs if e == 0 else decoders[-1]]))

    #return = [tf.keras.Model(inputs=[inputs], outputs=[dec]) for dec in decoders]
    return tf.keras.Model(inputs=[inputs], outputs=decoders)

In [89]:
import sys
sys.path.append('../') #root path
from functools import partial

class Model_Train():
    def __init__(self, summary_dir, num_metrics, checkpoint_dir, learning_rate, min_learning_rate):
        self.step = tf.Variable(0,dtype=tf.int64)
        self.num_metrics = num_metrics
        self.checkpoint_dir = checkpoint_dir
        self.learning_rate = learning_rate
        self.min_learning_rate = min_learning_rate 
        self.build_model()
        log_dir = os.path.join(summary_dir)
        self.train_summary_writer = tf.summary.create_file_writer(log_dir)

    def build_model(self):
        self.generator = S_Net_progressiveskip(num_metrics=self.num_metrics, structure_type='advanced')

        #Learning rate decay for every 200 iterations
        self.lr_scheduler_fn =  tf.compat.v1.train.exponential_decay(self.learning_rate, self.step, 200, 0.1,  staircase=True)
        self.learning_rate = lambda : tf.maximum(self.min_learning_rate, self.lr_scheduler_fn())

        self.generator_optimizer = tf.keras.optimizers.Adam( self.learning_rate )

        """ saver """
        self.ckpt = tf.train.Checkpoint(step=self.step,
                                        generator_optimizer=self.generator_optimizer,
                                        generator=self.generator,
                                        )
        self.save_manager = tf.train.CheckpointManager(self.ckpt, self.checkpoint_dir, max_to_keep=3)
        self.save  = lambda : self.save_manager.save(checkpoint_number=self.step) #exaple : model.save()



    @tf.function
    def training(self, inputs):
        paired_input, paired_target = inputs
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            predictions = self.generator(paired_input)
            losses = [L1loss(paired_target, predictions[i]) for i in range(self.num_metrics)]
            total_loss = tf.reduce_mean(losses)

        """ optimize """
        params_gradients = self.generator.trainable_variables
        generator_gradients = gen_tape.gradient(total_loss, params_gradients)
        self.generator_optimizer.apply_gradients(zip(generator_gradients, params_gradients))

        inputs_concat = tf.concat([paired_input, paired_target], axis=2)
        return_dicts = {"inputs_concat" :inputs_concat}
        return_dicts.update({'total_loss{}'.format(e) : l  for e,l in enumerate(losses)})
        return_dicts.update({'total_loss' : total_loss})
        return_dicts.update({'Predictions{}'.format(e) : tf.concat([paired_input,l,paired_target],axis=2)  for e,l in enumerate(predictions)})
        return return_dicts



    def train_step(self,iterator, summary_name = "train", log_interval = 100):
        """ training """
        result_logs_dict = self.training(iterator.__next__())

        """ log summary """
        if summary_name and self.step.numpy() % log_interval == 0:
            with self.train_summary_writer.as_default():
                for key, value in result_logs_dict.items():
                    value = value.numpy()
                    if len(value.shape) == 0:
                        tf.summary.scalar("{}_{}".format(summary_name,key), value, step=self.step)
                    elif len(value.shape) in [3,4]:
                        tf.summary.image("{}_{}".format(summary_name, key), denormalize(value), step=self.step)


        """ return log str """
        log = "Total_Loss : {} lr : {}".format(result_logs_dict["total_loss"], self.learning_rate().numpy())
        return log, [denormalize(result_logs_dict["Predictions{}".format(i)].numpy() )for i in range(self.num_metrics)]




    # Typically, the test dataset is not large
    @tf.function
    def inference(self, input_image):
        return self.generator(input_image)

    def test_step(self, test_dataset, summary_name = "test"):
        outputs = [[] for _ in range(self.num_metrics)]
        losses = [[] for _ in range(self.num_metrics)]
        PSNRs = [[] for _ in range(self.num_metrics)]
        SSIMs = [[] for _ in range(self.num_metrics)]

        for input_image_test,label_image_test in test_dataset:
            predictions = self.inference(input_image_test)
            for e, pred in enumerate(predictions):
                losses[e].append(L1loss(label_image_test, pred).numpy())
                outputs[e].append(np.concatenate([input_image_test,pred.numpy(),label_image_test],axis=2))
                crop_pad = 8
                A = tf.split(tf.image.rgb_to_yuv(tf.convert_to_tensor(label_image_test[:,crop_pad:-crop_pad - 1, crop_pad:-crop_pad - 1] )), [1,2],-1)[0]
                B = tf.split(tf.image.rgb_to_yuv(tf.convert_to_tensor(pred.numpy()[:,crop_pad:-crop_pad - 1, crop_pad:-crop_pad - 1] )), [1,2],-1)[0]
                PSNRs[e].append(tf.image.psnr(A,B,1).numpy())
                SSIMs[e].append(tf.image.ssim(A,B,1).numpy())

        """ log summary """
        if summary_name and self.step.numpy() %100 == 0:
            with self.train_summary_writer.as_default():
                for e, output in enumerate(outputs):
                    tf.summary.image("{}_pred_{}_0".format(summary_name,e), denormalize(output[0]), step=self.step)
                    tf.summary.image("{}_pred_{}_1".format(summary_name,e), denormalize(output[1]), step=self.step)
                for e, loss in enumerate(losses):
                    tf.summary.scalar("{}_loss_{}".format(summary_name, e),np.mean(loss), step=self.step)
                for e, PSNR in enumerate(PSNRs):
                    tf.summary.scalar("{}_psnr_{}".format(summary_name, e),np.mean(PSNR), step=self.step)
                for e, SSIM in enumerate(SSIMs):
                    tf.summary.scalar("{}_ssim_{}".format(summary_name, e),np.mean(SSIM), step=self.step)

        """ return log str """
        log = "\n"
        for i in range(self.num_metrics):
            log += "[output{}] loss = {}, psnr = {}, ssim = {}\n".format(i,np.mean(losses[i]),np.mean(PSNRs[i]),np.mean(SSIMs[i]))
        return log


In [90]:
!rm -rf SNET*

In [91]:
!rm -rf /content/outputs

In [92]:
!mkdir outputs

In [ ]:
import datetime
import time

start = time.time()
time_now = datetime.datetime.now()


def generate_expname_automatically():
    name = "SNET_%s_%02d_%02d_%02d_%02d_%02d" % ("assignment",
            time_now.month, time_now.day, time_now.hour,
            time_now.minute, time_now.second)
    return name

expname  = generate_expname_automatically()
checkpoint_dir = expname ; check_folder("./__outputs/checkpoints/")
summary_dir = expname ; check_folder("./__outputs/summary/")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


""" build model """
 
num_metrics = 8
learning_rate = 0.0001
min_learning_rate = 0.000001

model = Model_Train(summary_dir, num_metrics, checkpoint_dir, learning_rate, min_learning_rate)

""" restore model """
if False:
    model.ckpt.restore(config.restore_file)



jpeg_quality = 30
patch_size = 48
batch_size = 16
data_path_train="/content/train/train_y"
data_path_test="/content/validation/clean"

trainset_dispenser = Trainset_Dispenser(data_path_train,jpeg_quality, patch_size, batch_size)
testset_dispenser = Testset_Dispenser(data_path_test, jpeg_quality)
count = 0
steps = 0
while steps < 501: #manually stopping
    """ train """
    log, output = model.train_step(trainset_dispenser, log_interval= 100)
    if model.step.numpy() % 1 == 0:
        print("[train] step:{} elapse:{} {}".format(model.step.numpy(), time.time() - start, log))

        #visualization
        output_concat = np.concatenate([output[i] for i in range(len(output))], axis=1)[0]
        output_concat = cv2.resize(output_concat,(output_concat.shape[1]*3,output_concat.shape[0]*3))
        cv2.imwrite('outputs/image_'+str(count)+'_.png', output_concat[...,::-1])
        count+=1

    if model.step.numpy() % 100 == 0:
      log = model.test_step(testset_dispenser, summary_name="test")
      print("[test] step:{} elapse:{} {}".format(model.step.numpy(), time.time() - start, log))

    """ save model """
    if model.step.numpy() % 100 == 0:  save_path = model.save()
    model.step.assign_add(1)
    steps += 1

/usr/local/lib/python3.7/dist-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "
100%|██████████| 19/19 [00:00<00:00, 52.71it/s]
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Constant'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Inference

In [ ]:
!rm -rf /content/weights

In [ ]:
!mv SNET* weights

In [ ]:
num_metrics = 8
learning_rate = 0.0001
min_learning_rate = 0.000001

In [ ]:
import os
checkpoint_dir = "./__outputs/checkpoints/"
summary_dir = "./__outputs/summary/"

In [ ]:
def normalize(images):
    return (images.astype(np.float32)/255.0)

In [ ]:
def denormalize(images):
    return np.clip(images*255, a_min=0, a_max=255).astype(np.uint8)

In [ ]:
!mkdir preds

mkdir: cannot create directory ‘preds’: File exists


In [ ]:
!rm -rf /content/validation/down_grade/.ipynb_checkpoints

In [ ]:
from google.colab.patches import cv2_imshow
import cv2

def predict(image, model):
  image = np.array(image)
  ts = np.empty([image.shape[0], image.shape[1], 3], dtype=np.uint8)
  image = normalize(image)
  #cv2_imshow(cv2.cvtColor(ts, cv2.COLOR_BGR2RGB))
  for x in range(0,image.shape[0],48):
    for y in range(0,image.shape[1],48):
      im = image[x:x+48,y:y+48]
      im = np.expand_dims(im.astype("float32"), axis=0)
      outs = model.generator(im)
      output = denormalize(outs[-1])
      ex = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
      ts[x:x+48, y:y+48, :] = output
      #cv2_imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
      #cv2.imwrite("outs"+"/"+str(x)+str(y)+".jpg", cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
      cv2_imshow(cv2.cvtColor(ts, cv2.COLOR_BGR2RGB))
  return ts

In [ ]:
from PIL import Image 
import numpy as np

In [ ]:
model = Model_Train(summary_dir, num_metrics, checkpoint_dir, learning_rate, min_learning_rate)
model.ckpt.restore("/content/weights/ckpt-500")
for i in os.listdir("/content/validation/down_grade/"):
  image = Image.open("/content/validation/down_grade/"+i).convert('RGB')
  image_clean = Image.open("/content/validation/clean/"+i).convert('RGB')
  image_clean = np.array(image_clean)
  if image_clean.shape[0] % 48 != 0 or image_clean.shape[1] % 48 != 0:
    continue
  output = predict(image, model)
  im = Image.fromarray(output)
  print("\n PSNR for original/reconstructed:"+i,tf.image.psnr(image_clean, output, 255))
  print("\n SSIM for original/reconstructed:"+i,tf.image.ssim(tf.convert_to_tensor(image_clean),tf.convert_to_tensor(output),255))
  im.save("preds/"+i)
  image.save("preds/dg_"+i)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def predict(image, model):
  image = np.array(image)
  ts = np.empty([image.shape[0], image.shape[1], 3], dtype=image.dtype)
  for x in range(0,image.shape[0],48):
    for y in range(0,image.shape[1],48):
      im = image[x:x+48,y:y+48]
      ts[x:x+48, y:y+48, :] = im
  return ts

for i in os.listdir("/content/validation/down_grade/"):
  image = Image.open("/content/validation/down_grade/"+i).convert('RGB')
  image_clean = Image.open("/content/validation/clean/"+i).convert('RGB')
  image_clean = np.array(image_clean)
  if image_clean.shape[0] % 70 != 0 or image_clean.shape[1] % 70 != 0:
    continue
  output = predict(image, model)
  im = Image.fromarray(output.astype(np.uint8))
  display(im)

In [ ]:
!zip -r outputs.zip outputs

In [ ]:
image_clean.shape

(288, 288, 3)

In [ ]:
!git clone https://github.com/jiaxi-jiang/FBCNN.git

Cloning into 'FBCNN'...
remote: Enumerating objects: 201, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 201 (delta 20), reused 6 (delta 4), pack-reused 174
Receiving objects: 100% (201/201), 46.28 MiB | 18.14 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [ ]:
%cd FBCNN

/content/FBCNN


In [ ]:
!python main_train_fbcnn.py

export CUDA_VISIBLE_DEVICES=0
LogHandlers setup!
22-08-29 19:27:43.785 :   task: FBCNN-Color
  model: fbcnn
  gpu_ids: [0]
  n_channels: 3
  merge_bn: False
  merge_bn_startpoint: 40000000
  path:[
    root: deblocking
    pretrained_netG: None
    task: deblocking/FBCNN-Color
    log: deblocking/FBCNN-Color
    options: deblocking/FBCNN-Color/options
    models: deblocking/FBCNN-Color/models
    images: deblocking/FBCNN-Color/images
  ]
  datasets:[
    train:[
      name: train_dataset
      dataset_type: jpeg
      dataroot_H: /content/train/train_y
      dataroot_L: None
      H_size: 96
      dataloader_shuffle: True
      dataloader_num_workers: 16
      dataloader_batch_size: 16
      phase: train
      scale: 1
      n_channels: 3
    ]
    test:[
      name: test_dataset
      dataset_type: jpeg
      dataroot_H: /content/validation/clean
      dataroot_L: /content/validation/down_grade
      phase: test
      scale: 1
      n_channels: 3
    ]
  ]
  netG:[
    net_type: fbcnn

In [ ]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/nightly/cu100/torch_nightly.html
     |████████████████████████████████| 790.0 MB 20 kB/s 


In [ ]:
!pip uninstall torchvision
!pip install torchvision==0.4.0 -f https://download.pytorch.org/whl/torch_stable.html

Found existing installation: torchvision 0.13.1+cu113
Uninstalling torchvision-0.13.1+cu113:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/torchvision-0.13.1+cu113.dist-info/*
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libcudart.053364c0.so.11.0
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libnvjpeg.90286a3c.so.11
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /usr/local/lib/python3.7/dist-packages/torchvision.libs/libz.1328edc3.so.1
    /usr/local/lib/python3.7/dist-packages/torchvision/*
Proceed (y/n)? y
  Successfully uninstalled torchvision-0.13.1+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 8.8 MB 49.2 MB/s 
     |████████████████████████████████| 6